
# MNIST свёрточными сетями

## Читаем данные

In [1]:
from packaging import version

import numpy as np
import tensorflow as tf
import keras
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")
sns.set_palette("colorblind")
palette = sns.color_palette()
figsize = (10, 10)
legend_fontsize = 16

assert version.parse(tf.__version__) >= version.parse('1.0.0'), 'need TensorFlow 1.0.0 or greater, got %s' % tf.__version__
assert version.parse(keras.__version__) >= version.parse('2.0.0'), 'need Keras 2.0.0 or greater, got %s' % keras.__version__

tf.__version__, keras.__version__

Using TensorFlow backend.


('1.0.0', '2.0.4')

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## Свёртки на TensorFlow

In [3]:
x = tf.placeholder(tf.float32, [None, 784]) 
y = tf.placeholder(tf.float32, [None, 10])
keep_probability = tf.placeholder(tf.float32)

x_image = tf.reshape(x, [-1, 28, 28, 1])

W_conv_0 = tf.Variable(tf.truncated_normal([5, 5, 1, 32], stddev=0.1))
b_conv_0 = tf.Variable(tf.constant(0.1, shape=[32]))

conv_0 = tf.nn.conv2d(x_image, W_conv_0, strides=[1, 1, 1, 1], padding='SAME') + b_conv_0

h_conv_0 = tf.nn.relu(conv_0)
h_pool_0 = tf.nn.max_pool(h_conv_0, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
h_pool_1_flat = tf.reshape(h_pool_0, [-1, 14 * 14 * 32])

W_fc_1 = tf.Variable(tf.truncated_normal([14 * 14 * 32, 10], stddev=0.1))
b_fc_1 = tf.Variable(tf.constant(0.1, shape=[10]))

logit_conv = tf.matmul(h_pool_1_flat, W_fc_1) + b_fc_1
y_conv = tf.nn.softmax(logit_conv)

In [4]:
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

l2 = tf.contrib.layers.l2_regularizer(0.1)
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logit_conv, labels=y)
) + l2(W_conv_0)  # + l2(W_conv_1)# + l1(W_conv_0) + l1(W_conv_1) + l1(W_conv_2)

train_step = tf.train.AdamOptimizer(0.0001).minimize(cross_entropy)

In [5]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
%%time
for i in range(20001):
    batch_xs, batch_ys = mnist.train.next_batch(64)
    sess.run(train_step, feed_dict={x: batch_xs, y: batch_ys, keep_probability: 0.5})
    if i % 50 == 0:
        test_xs, test_ys = mnist.test.next_batch(128)
        acc = sess.run(accuracy, feed_dict={x: test_xs, y: test_ys, keep_probability: 1.})
        print("Точность %s: %s" % (i, acc))
    if i > 0 and i % 1000 == 0:
        print("[%d]\ttrain accuracy=%.6f\ttest accuracy=%.6f" % (i,
            sess.run(accuracy, feed_dict={x: mnist.train.images, y: mnist.train.labels}),
            sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels})
        ))

Точность 0: 0.09375
Точность 50: 0.1875
Точность 100: 0.453125
Точность 150: 0.625
Точность 200: 0.625
Точность 250: 0.742188
Точность 300: 0.828125
Точность 350: 0.8125
Точность 400: 0.820312
Точность 450: 0.734375
Точность 500: 0.804688
Точность 550: 0.796875
Точность 600: 0.867188
Точность 650: 0.8125
Точность 700: 0.882812
Точность 750: 0.804688
Точность 800: 0.820312
Точность 850: 0.875
Точность 900: 0.8125
Точность 950: 0.882812
Точность 1000: 0.859375
[1000]	train accuracy=0.886273	test accuracy=0.894000
Точность 1050: 0.890625
Точность 1100: 0.914062
Точность 1150: 0.90625
Точность 1200: 0.851562
Точность 1250: 0.898438
Точность 1300: 0.890625
Точность 1350: 0.882812
Точность 1400: 0.9375
Точность 1450: 0.820312
Точность 1500: 0.859375
Точность 1550: 0.890625
Точность 1600: 0.859375
Точность 1650: 0.851562
Точность 1700: 0.898438
Точность 1750: 0.90625
Точность 1800: 0.945312
Точность 1850: 0.898438
Точность 1900: 0.867188
Точность 1950: 0.90625
Точность 2000: 0.945312
[2000]	t

## Двухуровневая свёрточная сеть

In [11]:
x = tf.placeholder(tf.float32, [None, 784]) 
y = tf.placeholder(tf.float32, [None, 10])
keep_probability = tf.placeholder(tf.float32)

x_image = tf.reshape(x, [-1, 28,28 ,1])

W_conv_0 = tf.Variable(tf.truncated_normal([5, 5, 1, 32], stddev=0.1))
b_conv_0 = tf.Variable(tf.constant(0.1, shape=[32]))

conv_0 = tf.nn.conv2d(x_image, W_conv_0, strides=[1, 1, 1, 1], padding='SAME') + b_conv_0

h_conv_0 = tf.nn.relu(conv_0)
h_pool_0 = tf.nn.max_pool(h_conv_0, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

W_conv_1 = tf.Variable(tf.truncated_normal([5, 5, 32, 64], stddev=0.1))
b_conv_1 = tf.Variable(tf.constant(0.1, shape=[64]))

conv_1 = tf.nn.conv2d(h_pool_0, W_conv_1, strides=[1, 1, 1, 1], padding='SAME') + b_conv_1

h_conv_1 = tf.nn.relu(conv_1)
h_pool_1 = tf.nn.max_pool(h_conv_1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
h_pool_1_flat = tf.reshape(h_pool_1, [-1, 7 * 7 * 64])

W_fc_2 = tf.Variable(tf.truncated_normal([7 * 7 * 64, 10], stddev=0.1))
b_fc_2 = tf.Variable(tf.constant(0.1, shape=[10]))

logit_conv = tf.matmul(h_pool_1_flat, W_fc_2) + b_fc_2
y_conv = tf.nn.softmax(logit_conv)

In [15]:
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

l2 = tf.contrib.layers.l2_regularizer(0.1)
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logit_conv, y))
# +                   l2(W_conv_0) #+ l2(W_conv_1)# + l1(W_conv_0) + l1(W_conv_1) + l1(W_conv_2)

train_step = tf.train.AdamOptimizer().minimize(cross_entropy)

In [13]:
sess = tf.Session()
sess.run(tf.initialize_all_variables())

In [14]:
for i in range(20001):
    batch_xs, batch_ys = mnist.train.next_batch(64)
    sess.run(train_step, feed_dict={x: batch_xs, y: batch_ys, keep_probability: 0.5})
    if i % 50 == 0:
        test_xs, test_ys = mnist.test.next_batch(128)
        acc = sess.run(accuracy, feed_dict={x: test_xs, y: test_ys, keep_probability: 1.})
        print("Точность %s: %s" % (i, acc))
    if i > 0 and i % 1000 == 0:
        print("[%d]\ttest accuracy=%.6f" % (i,
#             sess.run(accuracy, feed_dict={x: mnist.train.images, y: mnist.train.labels}),
            sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels})
        ))

Точность 0: 0.101562
Точность 50: 0.875
Точность 100: 0.9375
Точность 150: 0.945312
Точность 200: 0.9375
Точность 250: 0.960938
Точность 300: 0.976562
Точность 350: 0.9375
Точность 400: 0.992188
Точность 450: 0.976562
Точность 500: 0.960938
Точность 550: 0.992188
Точность 600: 0.976562
Точность 650: 0.984375
Точность 700: 0.953125
Точность 750: 0.953125
Точность 800: 0.960938
Точность 850: 0.984375
Точность 900: 0.960938
Точность 950: 0.96875
Точность 1000: 0.96875
[1000]	test accuracy=0.978300
Точность 1050: 1.0
Точность 1100: 0.992188
Точность 1150: 1.0
Точность 1200: 0.984375
Точность 1250: 0.992188
Точность 1300: 0.984375
Точность 1350: 0.960938
Точность 1400: 0.976562
Точность 1450: 0.984375
Точность 1500: 0.953125
Точность 1550: 0.976562
Точность 1600: 0.984375
Точность 1650: 0.992188
Точность 1700: 0.96875
Точность 1750: 0.984375
Точность 1800: 0.984375
Точность 1850: 0.992188
Точность 1900: 0.976562
Точность 1950: 1.0
Точность 2000: 0.960938
[2000]	test accuracy=0.983600
Точнос